### Run the following chunk of code to import any necessary libraries or packages which are needed for the rest of the script.

In [7]:
import os as os
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv2
import tensorflow as tf
from scipy.ndimage import label
from skimage import measure

### Run the following chunk of code to extract all areas and eccentricities of regions identified as "adhered" by the phase 1 segmentation network after adjusting the color of the input images.

The user will need to change the base_Directory and network_Directory variable, as well as pay careful attention to the directory structure of the script.

In [8]:
base_Directory = ""
network_Directory = ""
blob_Sizes = []
blob_Eccentricities = []
channel_Directory = base_Directory + "Analysis/All_Images_For_Analysis/"
network_File_Path = network_Directory

network = tf.keras.models.load_model(network_File_Path)

def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

def recolor_Image(img):
    maximum = np.mean(img) + 4*np.std(img)
    minimum = np.mean(img) - 4*np.std(img)
    color_Adjusted_Image = (img - minimum)/(maximum - minimum)
    color_Adjusted_Image[color_Adjusted_Image < 0] = 0
    color_Adjusted_Image[color_Adjusted_Image > 1] = 1
    return color_Adjusted_Image


for image_Name in os.listdir(channel_Directory):
    print("Analyzing " + image_Name[:-4])
    full_Channel = plt.imread(channel_Directory + image_Name)
    if np.max(full_Channel) == int(np.max(full_Channel)) and len(str(np.max(full_Channel))) == len(str(int(np.max(full_Channel)))):
        full_Channel = full_Channel.copy()/255.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel.copy(), cv2.COLOR_GRAY2RGB)
    if np.shape(full_Channel)[2] == 4:
        full_Channel = full_Channel.copy()[:,:,0:3]
    full_Channel = recolor_Image(full_Channel.copy())
    image_Height, image_Width, channels = np.shape(full_Channel)
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))
    x_Slider = 0
    y_Slider = 0
    output_Array = np.zeros((128,128))
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            output = network.predict(current_Tile_Normalized, verbose = 0)

            for i in range(128):
                for j in range(128):
                    output_Array[i,j] = np.argmax(output[0,i,j,:])
            
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    for i in range(image_Height_Resized):
        for j in range(image_Width_Resized):
            if output_Image[i,j] != 0:
                output_Image[i,j] = 1
            else:
                continue

    blobs, number_Of_Blobs = label(output_Image)
    properties = measure.regionprops(blobs)
    for prop in properties:
        blob_Sizes.append(prop.area)
        blob_Eccentricities.append(prop.eccentricity)
    print("======================================")

Analyzing 1
Analyzing 10
Analyzing 100
Analyzing 101
Analyzing 102
Analyzing 103
Analyzing 104
Analyzing 105
Analyzing 106
Analyzing 107
Analyzing 108
Analyzing 109
Analyzing 11
Analyzing 110
Analyzing 111
Analyzing 112
Analyzing 113
Analyzing 114
Analyzing 115
Analyzing 116
Analyzing 117
Analyzing 118
Analyzing 119
Analyzing 12
Analyzing 120
Analyzing 121
Analyzing 122
Analyzing 123
Analyzing 124
Analyzing 125
Analyzing 126
Analyzing 127
Analyzing 128
Analyzing 129
Analyzing 13
Analyzing 130
Analyzing 131
Analyzing 132
Analyzing 133
Analyzing 134
Analyzing 135
Analyzing 136
Analyzing 137
Analyzing 138
Analyzing 139
Analyzing 14
Analyzing 140
Analyzing 141
Analyzing 142
Analyzing 143
Analyzing 144
Analyzing 145
Analyzing 146
Analyzing 147
Analyzing 148
Analyzing 149
Analyzing 15
Analyzing 150
Analyzing 151
Analyzing 152
Analyzing 153
Analyzing 154
Analyzing 155
Analyzing 156
Analyzing 157
Analyzing 158
Analyzing 159
Analyzing 16
Analyzing 160
Analyzing 161
Analyzing 162
Analyzing 163
A

Analyzing 24
Analyzing 240
Analyzing 241
Analyzing 242
Analyzing 243
Analyzing 244
Analyzing 245
Analyzing 246
Analyzing 247
Analyzing 248
Analyzing 249
Analyzing 25
Analyzing 250
Analyzing 251
Analyzing 252
Analyzing 253
Analyzing 254
Analyzing 255
Analyzing 256
Analyzing 257
Analyzing 258
Analyzing 259
Analyzing 26
Analyzing 260
Analyzing 261
Analyzing 262
Analyzing 263
Analyzing 264
Analyzing 265
Analyzing 266
Analyzing 267
Analyzing 268
Analyzing 269
Analyzing 27
Analyzing 270
Analyzing 271
Analyzing 272
Analyzing 273
Analyzing 274
Analyzing 275
Analyzing 276
Analyzing 277
Analyzing 278
Analyzing 279
Analyzing 28
Analyzing 280
Analyzing 281
Analyzing 282
Analyzing 283
Analyzing 284
Analyzing 285
Analyzing 286
Analyzing 287
Analyzing 288
Analyzing 289
Analyzing 29
Analyzing 290
Analyzing 291
Analyzing 292
Analyzing 293
Analyzing 294
Analyzing 295
Analyzing 296
Analyzing 297
Analyzing 298
Analyzing 299
Analyzing 3
Analyzing 30
Analyzing 300
Analyzing 301
Analyzing 302
Analyzing 303
A

Analyzing 38
Analyzing 380
Analyzing 381
Analyzing 382
Analyzing 383
Analyzing 384
Analyzing 385
Analyzing 386
Analyzing 387
Analyzing 388
Analyzing 389
Analyzing 39
Analyzing 390
Analyzing 391
Analyzing 392
Analyzing 393
Analyzing 394
Analyzing 395
Analyzing 396
Analyzing 397
Analyzing 398
Analyzing 399
Analyzing 4
Analyzing 40
Analyzing 400
Analyzing 401
Analyzing 402
Analyzing 403
Analyzing 404
Analyzing 405
Analyzing 406
Analyzing 407
Analyzing 408
Analyzing 409
Analyzing 41
Analyzing 410
Analyzing 411
Analyzing 412
Analyzing 413
Analyzing 414
Analyzing 415
Analyzing 416
Analyzing 417
Analyzing 418
Analyzing 419
Analyzing 42
Analyzing 420
Analyzing 421
Analyzing 422
Analyzing 423
Analyzing 424
Analyzing 425
Analyzing 426
Analyzing 427
Analyzing 428
Analyzing 429
Analyzing 43
Analyzing 430
Analyzing 431
Analyzing 432
Analyzing 433
Analyzing 434
Analyzing 435
Analyzing 436
Analyzing 437
Analyzing 438
Analyzing 439
Analyzing 44
Analyzing 440
Analyzing 441
Analyzing 442
Analyzing 443
A

Analyzing 519
Analyzing 52
Analyzing 520
Analyzing 521
Analyzing 522
Analyzing 523
Analyzing 524
Analyzing 525
Analyzing 526
Analyzing 527
Analyzing 528
Analyzing 529
Analyzing 53
Analyzing 530
Analyzing 531
Analyzing 532
Analyzing 533
Analyzing 534
Analyzing 535
Analyzing 536
Analyzing 537
Analyzing 538
Analyzing 539
Analyzing 54
Analyzing 540
Analyzing 541
Analyzing 542
Analyzing 543
Analyzing 544
Analyzing 545
Analyzing 546
Analyzing 547
Analyzing 548
Analyzing 549
Analyzing 55
Analyzing 550
Analyzing 551
Analyzing 552
Analyzing 553
Analyzing 554
Analyzing 555
Analyzing 556
Analyzing 557
Analyzing 558
Analyzing 559
Analyzing 56
Analyzing 560
Analyzing 561
Analyzing 562
Analyzing 563
Analyzing 564
Analyzing 565
Analyzing 566
Analyzing 567
Analyzing 568
Analyzing 569
Analyzing 57
Analyzing 570
Analyzing 571
Analyzing 572
Analyzing 573
Analyzing 574
Analyzing 575
Analyzing 576
Analyzing 577
Analyzing 578
Analyzing 579
Analyzing 58
Analyzing 580
Analyzing 581
Analyzing 582
Analyzing 583

Analyzing 659
Analyzing 66
Analyzing 660
Analyzing 661
Analyzing 662
Analyzing 663
Analyzing 664
Analyzing 665
Analyzing 666
Analyzing 667
Analyzing 668
Analyzing 669
Analyzing 67
Analyzing 670
Analyzing 671
Analyzing 672
Analyzing 673
Analyzing 674
Analyzing 675
Analyzing 676
Analyzing 677
Analyzing 678
Analyzing 679
Analyzing 68
Analyzing 680
Analyzing 681
Analyzing 682
Analyzing 683
Analyzing 684
Analyzing 685
Analyzing 686
Analyzing 687
Analyzing 688
Analyzing 689
Analyzing 69
Analyzing 690
Analyzing 691
Analyzing 692
Analyzing 693
Analyzing 694
Analyzing 695
Analyzing 696
Analyzing 697
Analyzing 698
Analyzing 699
Analyzing 7
Analyzing 70
Analyzing 700
Analyzing 701
Analyzing 702
Analyzing 703
Analyzing 704
Analyzing 705
Analyzing 706
Analyzing 707
Analyzing 708
Analyzing 709
Analyzing 71
Analyzing 710
Analyzing 711
Analyzing 712
Analyzing 713
Analyzing 714
Analyzing 715
Analyzing 716
Analyzing 717
Analyzing 718
Analyzing 719
Analyzing 72
Analyzing 720
Analyzing 721
Analyzing 722
A

Analyzing 799
Analyzing 8
Analyzing 80
Analyzing 800
Analyzing 801
Analyzing 802
Analyzing 803
Analyzing 804
Analyzing 805
Analyzing 806
Analyzing 807
Analyzing 808
Analyzing 809
Analyzing 81
Analyzing 810
Analyzing 811
Analyzing 812
Analyzing 813
Analyzing 814
Analyzing 815
Analyzing 816
Analyzing 817
Analyzing 818
Analyzing 819
Analyzing 82
Analyzing 820
Analyzing 821
Analyzing 822
Analyzing 823
Analyzing 824
Analyzing 825
Analyzing 826
Analyzing 827
Analyzing 828
Analyzing 829
Analyzing 83
Analyzing 830
Analyzing 831
Analyzing 832
Analyzing 833
Analyzing 834
Analyzing 835
Analyzing 836
Analyzing 837
Analyzing 838
Analyzing 839
Analyzing 84
Analyzing 840
Analyzing 841
Analyzing 842
Analyzing 843
Analyzing 844
Analyzing 845
Analyzing 846
Analyzing 847
Analyzing 848
Analyzing 849
Analyzing 85
Analyzing 850
Analyzing 851
Analyzing 852
Analyzing 853
Analyzing 854
Analyzing 855
Analyzing 856
Analyzing 857
Analyzing 858
Analyzing 859
Analyzing 86
Analyzing 860
Analyzing 861
Analyzing 862
A

Analyzing 938
Analyzing 939
Analyzing 94
Analyzing 940
Analyzing 941
Analyzing 942
Analyzing 943
Analyzing 944
Analyzing 945
Analyzing 946
Analyzing 947
Analyzing 95
Analyzing 96
Analyzing 97
Analyzing 98
Analyzing 99


### Run the following code to save the extracted areas and eccentricities as .npy numpy arrays.

In [9]:
np.save(base_Directory + "Analysis/Extracted_Morphological_Features/Region_Sizes.npy",blob_Sizes)
np.save(base_Directory + "Analysis/Extracted_Morphological_Features/Region_Eccentricities.npy",blob_Eccentricities)